# Smith-normal-form
**Definition.**
The Smith-normal form of a matrix $A$ over a principal ideal domain (PID) is a matrix $D$ 
$$
\begin{pmatrix}
\alpha_1 & 0      & \cdots   & \cdots & \cdots & 0     \\
0        & \ddots & \ddots   &        &        & \vdots\\
\vdots   & \ddots & \alpha_k & \ddots &        & \vdots\\
\vdots   &        & \ddots   & 0      & \ddots & \vdots\\
\vdots   &        &          & \ddots & \ddots & \vdots\\
0        & \cdots & \cdots   & \cdots & \cdots & 0
\end{pmatrix}
$$
such that $D = P * A * Q$ with $P$ & $Q$ invertible matrices and $\alpha_{i-1} | \alpha_i$ for $i \in \{1,\dots,k\}$.

To get a feeling for the algortihm for computing the Smith-normal-form let us consider the following example matrix

$$
\begin{pmatrix}
2 & 0 & 1 \\
4 & 5 & 2 \\
3 & 7 & 0 
\end{pmatrix}
$$

Let us define it with entries in the PID $\mathbb{Z}$

In [2]:
A = matrix( ZZ, [[2, 0, 1],[4, 5, 2],[3, 7, 0]])

Now let us walk through the algortihm step by step. 
We have defined our matrix A and now we need two matrices P,Q to keep track of our opperatrions performed on the matrix A


In [3]:
P = matrix.identity(A.base_ring(), A.nrows())
Q = matrix.identity(A.base_ring(), A.ncols())

The algorithm is a iterativ process starting with the choice of a Pivot element. This element must not be zero

In [4]:
def pivotChoice(P, A, Q, t):
    for i in range(t+1, min(A.ncols(),A.nrows())):
        if(A[i,i] != 0):
            return i, P, A, Q
        else:
            for j in range(0, A.nrows()):
                if(A[j,i] != 0):
                    A.swap_rows(i,j)
                    P.swap_rows(i,j)
                    return i, P, A, Q
                
    return -1, P, A, Q

In [5]:
pivot, P, A1, Q = pivotChoice(P, A, Q, -1)
show(P, A1, Q)
pivot

[1 0 0]
[0 1 0]
[0 0 1] [2 0 1]
[4 5 2]
[3 7 0] [1 0 0]
[0 1 0]
[0 0 1]

0

As shown by the example, our pivot position is 0 and nothing changed because the entry $a_{1,1}$ was not zero

We now get to the step were we check if our pivot element is devided by all entrys in that column. If this is not the case for an element then we replace our pivot by the greatest common divisor (gcd) of both elements. This is realised through the use of the extended euclidean algorithm (xgcd) and a multiplication by an invretable matrix $E$.

Later in the algorithm we perform the same step but this time for all elements in the row of the pivot

In [6]:
def pivotImproveRow(P, A, Q, i):
    for j in range(0, A.nrows()):
        if(not(A[i,i].divides(A[j,i]))):
            gcd, alpha, beta = xgcd(A[i,i],A[j,i])
            E = matrix.identity(A.base_ring(), A.nrows())
            E[i,i] = alpha
            E[i,j] = beta
            E[j,i] = -A[j,i]/gcd
            E[j,j] = A[i,i]/gcd
            A = E*A
            P = E*P
            j = 0
    return P, A, Q

def pivotImproveCol(P, A, Q, i):
    for j in range(0, A.ncols()):
        if(not(A[i,i].divides(A[i,j]))):
            gcd, alpha, beta = xgcd(A[i,i],A[i,j])
            E = matrix.identity(A.base_ring(), A.ncols())
            E[i,i] = alpha
            E[j,i] = beta
            E[i,j] = -A[i,j]/gcd
            E[j,j] = A[i,i]/gcd
            A = A*E
            Q = Q*E
            j = 0
    return P, A, Q

In [7]:
P, A1, Q = pivotImproveRow(P, A1, Q, pivot)
show(P, A1, Q)

[-1  0  1]
[ 0  1  0]
[-3  0  2] [ 1  7 -1]
[ 4  5  2]
[ 0 14 -3] [1 0 0]
[0 1 0]
[0 0 1]

As we now can see, there are some changes to our matirx $A$, which are also documented in our matrix $P$. Now the entry $a_{1,1}$ divides all entrys in this column

For the next step we eliminate all entrys in the column of the pivot, which are not the pivot.

In [8]:
def eliminateRow(P, A, Q, i):
    for j in range(0, A.nrows()):
        if(j != i):
            P.add_multiple_of_row(j,i,-A[j,i]/A[i,i])
            A.add_multiple_of_row(j,i,-A[j,i]/A[i,i])
    return P, A, Q

def eliminateCol(P, A, Q, i):
    for j in range(0, A.ncols()):
        if(j != i):
            Q.add_multiple_of_column(j,i,-A[i,j]/A[i,i])
            A.add_multiple_of_column(j,i,-A[i,j]/A[i,i])
    return P, A, Q

In [9]:
P, A1, Q = eliminateRow(P, A1, Q, pivot)
show(P, A1, Q)

[-1  0  1]
[ 4  1 -4]
[-3  0  2] [  1   7  -1]
[  0 -23   6]
[  0  14  -3] [1 0 0]
[0 1 0]
[0 0 1]

Now all entrys in the first column are zero except our pivot.

The next step is repeating the process for the row of the pivot
and if this leads to entrys in the column of the pivot to change to no zero entrys to start over at the column step.
this cycle is repeated until all entrys in the column and row of the pivot are zero exept for the pivot.

In [10]:
P, A1, Q = pivotImproveCol(P, A1, Q, pivot)
show(P, A1, Q)
P, A1, Q = eliminateCol(P, A1, Q, pivot)
show(P, A1, Q)

[-1  0  1]
[ 4  1 -4]
[-3  0  2] [  1   7  -1]
[  0 -23   6]
[  0  14  -3] [1 0 0]
[0 1 0]
[0 0 1]

[-1  0  1]
[ 4  1 -4]
[-3  0  2] [  1   0   0]
[  0 -23   6]
[  0  14  -3] [ 1 -7  1]
[ 0  1  0]
[ 0  0  1]

After this process is finished, a new pivot is chosen form the submatrix starting at $row = pivot+1$ and $column = pivot+1$.
This is now repeated until all entrys in the submatrix are zero or the starting row or column of the submatrix are larger than the rows and columns of the original matrix.

In [11]:
pivot, P, A1, Q = pivotChoice(P, A1, Q, pivot)
show(P, A1, Q)
show(pivot)
P, A1, Q = pivotImproveRow(P, A1, Q, pivot)
show(P, A1, Q)
P, A1, Q = eliminateRow(P, A1, Q, pivot)
show(P, A1, Q)
P, A1, Q = pivotImproveCol(P, A1, Q, pivot)
show(P, A1, Q)
P, A1, Q = eliminateCol(P, A1, Q, pivot)
show(P, A1, Q)

[-1  0  1]
[ 4  1 -4]
[-3  0  2] [  1   0   0]
[  0 -23   6]
[  0  14  -3] [ 1 -7  1]
[ 0  1  0]
[ 0  0  1]

1

[ -1   0   1]
[ -3   3  -2]
[ 13 -14  10] [  1   0   0]
[  0   1   3]
[  0   0 -15] [ 1 -7  1]
[ 0  1  0]
[ 0  0  1]

[ -1   0   1]
[ -3   3  -2]
[ 13 -14  10] [  1   0   0]
[  0   1   3]
[  0   0 -15] [ 1 -7  1]
[ 0  1  0]
[ 0  0  1]

[ -1   0   1]
[ -3   3  -2]
[ 13 -14  10] [  1   0   0]
[  0   1   3]
[  0   0 -15] [ 1 -7  1]
[ 0  1  0]
[ 0  0  1]

[ -1   0   1]
[ -3   3  -2]
[ 13 -14  10] [  1   0   0]
[  0   1   0]
[  0   0 -15] [ 1 -7 22]
[ 0  1 -3]
[ 0  0  1]

We now have a matrix in the form of D but we still need to make sure that all $\alpha_{i-1}$ divide $\alpha_i$. This is done in the normalization step. In this step we also make sure that there are no gaps between the $\alpha_i$.

In [12]:
def normalize(P, A, Q):
    d = A.nonzero_positions()
    improved = true
    while(improved):
        improved = false
        for i in range (len(d)):
            if(d[0][0] != 0 or (d[i-1][0]+1 != d[i][0] and i != 0)):
                A.swap_rows(d[i][0],d[i][0]-1)
                P.swap_rows(d[i][0],d[i][0]-1)
                A.swap_columns(d[i][0],d[i][0]-1)
                Q.swap_columns(d[i][0],d[i][0]-1) 
                d[i] = [d[i][0]-1,d[i][1]-1]
                i = 0
                improved = true
    
    d = A.nonzero_positions()
    improved = true
    while(improved):
        improved = false
        for i in range(1,len(d)):
            if(not(A[d[i-1]].divides(A[d[i]]))):
                improved = true
                A.add_multiple_of_row(d[i-1][0],d[i][0], 1)
                P.add_multiple_of_row(d[i-1][0],d[i][0], 1)
                P, A, Q = pivotImproveCol(P, A, Q, d[i-1][0])
                P, A ,Q = eliminateRow(P, A, Q, d[i-1][0])
                P, A ,Q = eliminateCol(P, A, Q, d[i-1][0])
                
        if(issubclass(type(A.base_ring()),sage.rings.polynomial.polynomial_ring.PolynomialRing_field)):
            for i in range(0, len(d)):
                if(A[d[i]].coefficients()[-1].is_unit() and A[d[i]] != 0):
                    P.rescale_row(d[i][0], 1/A[d[i]].coefficients()[-1])
                    A.rescale_row(d[i][0], 1/A[d[i]].coefficients()[-1])
                if(not(issubclass(type(A.base_ring()),sage.rings.polynomial.polynomial_ring.PolynomialRing_dense_finite_field))):
                    P.rescale_row(d[i][0], sgn(A[d[i]].coefficients()[-1]))
                    A.rescale_row(d[i][0], sgn(A[d[i]].coefficients()[-1]))
        else:
            for i in range(0, len(d)):
                if(A[d[i]].is_unit() and A[d[i]] != 0):
                    P.rescale_row(d[i][0], 1/A[d[i]])
                    A.rescale_row(d[i][0], 1/A[d[i]])
                if(not(issubclass(type(A.base_ring()),sage.rings.finite_rings.finite_field_givaro.FiniteField))):
                    P.rescale_row(d[i][0], sgn(A[d[i]]))
                    A.rescale_row(d[i][0], sgn(A[d[i]]))
    return P, A ,Q

In [13]:
P, D, Q = normalize(P, A1, Q)
show(P, D, Q)

[ -1   0   1]
[ -3   3  -2]
[-13  14 -10] [ 1  0  0]
[ 0  1  0]
[ 0  0 15] [ 1 -7 22]
[ 0  1 -3]
[ 0  0  1]

As you might have noticed, the entry $a_{3,3}$ changed from -15 to 15. This is something we do for readability and comparability. We also change all unit entrys to 1 for the same reasons. These two steps don't need to be taken because the Smith normal form is only unique up to multiplication by an unit.

If we now Test with our matrix $A$ the following Equation should be true: $ D = P * A * Q$

In [14]:
show(D)
show(P * A * Q)

[ 1  0  0]
[ 0  1  0]
[ 0  0 15]

[ 1  0  0]
[ 0  1  0]
[ 0  0 15]

The following code combines all steps we made individually into one function only requiring us to input the matrix $A$ for which we wish to compute the Smith normal form

In [15]:
def smithNormalForm(A):
    D = copy(A)
    Q = matrix.identity(D.base_ring(), D.ncols())
    P = matrix.identity(D.base_ring(), D.nrows())
    pivot = -1
    while(pivot < D.ncols()):
        pivot, P, D, Q = pivotChoice(P, D, Q, pivot)
        if(pivot == -1):
            break
        while(len(D.nonzero_positions_in_column(pivot)) > 1 or len(D.nonzero_positions_in_row(pivot)) > 1):
            P, D, Q = pivotImproveRow(P, D, Q, pivot)
            P, D, Q = eliminateRow(P, D, Q, pivot)
            P, D, Q = pivotImproveCol(P, D, Q, pivot)
            P, D, Q = eliminateCol(P, D, Q, pivot)  
      
    P, D, Q = normalize(P, D, Q)
    return P, D, Q

This algorithm is implemented over any PID with excat evaluation on a Computer, mainly not over $\mathbb{R}$ and $\mathbb{R}$[x].

The following will give some exampels with randomly generated matrices:

In [16]:
A = random_matrix(QQ, 4, 5)
P, D, Q = smithNormalForm(A)
show(A)
show(D)
show(P * A * Q)


[ -2 1/2  -1   2  -1]
[ -1   0  -2   0   0]
[  0   0  -2   0   2]
[  2   0  -2  -1  -1]

[1 0 0 0 0]
[0 1 0 0 0]
[0 0 1 0 0]
[0 0 0 1 0]

[1 0 0 0 0]
[0 1 0 0 0]
[0 0 1 0 0]
[0 0 0 1 0]

In [17]:
A = random_matrix(QQ['x'], 4, 7)
P, D, Q = smithNormalForm(A)
show(A)
show(D)
show(P * A * Q)

[             4/25*x + 1                       0                   -37*x                       0   1/4*x^2 - 383*x - 1/4             -1/2*x + 10                       2]
[             -4/3*x + 1                  -x + 1   101*x^2 + 1/3*x - 1/2                      -1                       2                       0                 1/2*x^2]
[-475*x^2 + 9/11*x + 3/2        -1/5*x^2 + 1/2*x            -1/3*x - 1/8                       0                   1/4*x               1/3*x - 1            5/41*x^2 + 5]
[           -2*x^2 + 1/2                    3/46                       1              11/5*x - 1            1/70*x^2 - 2                       0            -2*x^2 + 7/2]

[1 0 0 0 0 0 0]
[0 1 0 0 0 0 0]
[0 0 1 0 0 0 0]
[0 0 0 1 0 0 0]

[1 0 0 0 0 0 0]
[0 1 0 0 0 0 0]
[0 0 1 0 0 0 0]
[0 0 0 1 0 0 0]

In [18]:
A = random_matrix(GF(3), 6, 2)
P, D, Q = smithNormalForm(A)
show(A)
show(D)
show(P * A * Q)

[2 0]
[2 2]
[0 0]
[1 2]
[0 0]
[1 1]

[1 0]
[0 1]
[0 0]
[0 0]
[0 0]
[0 0]

[1 0]
[0 1]
[0 0]
[0 0]
[0 0]
[0 0]

In [19]:
A = random_matrix(GF(7)['x'], 10, 5)
P, D, Q = smithNormalForm(A)
show(A)
show(D)
show(P * A * Q)

[              0               2 6*x^2 + 5*x + 5   x^2 + 2*x + 1         5*x + 6]
[              0               4             4*x               0               0]
[        3*x + 1         6*x + 5             3*x               0         5*x + 1]
[      2*x^2 + 6               3 5*x^2 + 6*x + 2         3*x + 5 4*x^2 + 3*x + 5]
[4*x^2 + 4*x + 3               3               3   6*x^2 + x + 4               2]
[              3         4*x + 2 2*x^2 + 3*x + 3               0               1]
[              0               6         3*x + 6               2         6*x + 5]
[              3               0               0 3*x^2 + 4*x + 1       5*x^2 + 6]
[              1         5*x + 3               2         3*x + 3           x + 6]
[              5         2*x + 5         5*x + 6           x + 4               0]

[1 0 0 0 0]
[0 1 0 0 0]
[0 0 1 0 0]
[0 0 0 1 0]
[0 0 0 0 1]
[0 0 0 0 0]
[0 0 0 0 0]
[0 0 0 0 0]
[0 0 0 0 0]
[0 0 0 0 0]

[1 0 0 0 0]
[0 1 0 0 0]
[0 0 1 0 0]
[0 0 0 1 0]
[0 0 0 0 1]
[0 0 0 0 0]
[0 0 0 0 0]
[0 0 0 0 0]
[0 0 0 0 0]
[0 0 0 0 0]

If you want to verify that the algortihm is correct you can use the function smith_form(), which is available in sage by default. You can use it like this:

In [20]:
A = random_matrix(ZZ, 5, 5)
P, D, Q = smithNormalForm(A)
D1, P1, Q1 = A.smith_form(A)
P1, D1, Q1 = normalize(P1, D1, Q1)
show(D)
show(D1)

[  1   0   0   0   0]
[  0   1   0   0   0]
[  0   0   1   0   0]
[  0   0   0   2   0]
[  0   0   0   0 958]

[  1   0   0   0   0]
[  0   1   0   0   0]
[  0   0   1   0   0]
[  0   0   0   2   0]
[  0   0   0   0 958]

As you can see, we still call normalize() on the Smith normal form provided by the sage function. This is due to the Smith normal form only being unique up to multipication by a unit. If we don't do this the result might look slightly different

In [21]:
A = random_matrix(QQ['x'], 3, 3)
P, D, Q = smithNormalForm(A)
D1, P1, Q1 = A.smith_form(A)
show(D)
show(D1)

[                        1                         0                         0]
[                        0                         1                         0]
[                        0                         0 x^4 - 1/20*x^3 + 3/80*x^2]

[                                1                                 0                                 0]
[                                0                                 1                                 0]
[                                0                                 0 -160/11*x^4 + 8/11*x^3 - 6/11*x^2]